In [47]:
import math
import copy
import numpy as np
from scipy.optimize import linprog

In [119]:
class Node():
    
    def __init__(self, size, costs, sortedEdges, allSortedEdges, parent_constr, extra_constr=None):
    
        self.size = size # Number of cities
        self.costs = costs # Distance matrix
        self.sortedEdges = sortedEdges
        self.allSortedEdges = allSortedEdges
        self.extra_constr = extra_constr
        self.constraints = self.determine_constr(parent_constr)
        self.lowerBound = self.LPLowerBound()
        self.LP = self.LPLowerBound()
        
    #This method calculates the simple lower bound (SB)
    #This is the sum over all vertices of the two smallest allowed edges
    def computeLowerBound(self):
        lb = 0
        for i in range(self.size):
            lower = 0
            t = 0
            for j in range(self.size):
                if self.constraints[i][j] == 1:
                    lower += self.costs[i][j]
                    t += 1
            tt = 0
            while t < 2:
                shortest = self.sortedEdges[i][tt]
                if self.constraints[i][shortest] == 2:
                    lower += self.costs[i][shortest]
                    t += 1
                tt += 1
                if tt >= self.size:
                    lower = math.inf
                    break
            lb += lower
        return lb

    def LPLowerBound(self):
        n = self.size
        
        edges = []
        for i in range(0, n):
            for j in range(i+1,n):
                edges += [self.costs[i][j]]
                
        A = np.array([[0]*(len(edges))]*n)
        for i in range(n):
            klist = [i]
            k = i
            for l in reversed(range(1, n-1)):
                klist += [k+l]
                k = k+l
            for j in range(int(((n-1)+(n-1-i))*(i+1)/2)-(n-1-i), int(((n-1)+(n-1-i))*(i+1)/2)):
                A[i][j] = 1
            for m in klist[0:i]:
                A[i][m-1] = 1
                
        b = [2]*n
        
        for i in range(n):
            for j in range(i+1, n):
                index = int(((n-1)+(n-i))*i/2 + (j-i-1))
                #print(self.constraints)
                if self.constraints[i][j] == 0:
                    A[i][index] = A[j][index] = 0
                if self.constraints[i][j] == 1:
                    A[i][index] = A[j][index] = n
                    b[i] += n-1
                    b[j] += n-1
        
        res = linprog(edges, A_eq=A, b_eq=b, bounds=(0,1), options={"disp": True})
        if res.get("success") == True:
            lp = res.get("fun")*2
            #print(res.get("x"))
        if res.get("success") == False:
            lp = math.inf
            
        return lp
    
    #This method determines the constraints of a node 
    #based on the parent constaints and the extra constraint
    def determine_constr(self, parent_constr):
        constraints = copy.deepcopy(parent_constr)
        if self.extra_constr == None:
            return constraints
        fr = self.extra_constr[0]
        to = self.extra_constr[1]
        constraints[fr][to] = self.extra_constr[2]
        constraints[to][fr] = self.extra_constr[2]
        for i in range(2):
            constraints = self.removeEdges(constraints)
            constraints = self.addEdges(constraints)
        
        return constraints

    #50
    #This method exclude edges when:
    #1. Two other edges adjacent to the same vertex are included
    #2. Including the edge would create a subtour
    def removeEdges(self, constraints):
        for i in range(self.size):
            t = 0
            for j in range(self.size):
                if (i != j) and (constraints[i][j] == 1):
                    t += 1
            if t >= 2:
                for j in range(self.size):
                    if constraints[i][j] == 2:
                        constraints[i][j] = 0
                        constraints[j][i] = 0
        for i in range(self.size):
            for j in range(self.size):
                t = 1
                prev = i
                fr = j
                cycle = False
                nextOne = self.next_one(prev, fr, constraints)
                while (nextOne[0]):
                    t += 1
                    next = nextOne[1]
                    if next == i:
                        cycle = True
                        break
                    if t > self.size:
                        break
                    prev = fr
                    fr = next
                    nextOne = self.next_one(prev, fr, constraints)
                if (cycle) and (t < self.size) and (constraints[i][j] == 2):
                    constraints[i][j] = 0
                    constraints[j][i] = 0
        return constraints
    
    #This method checks if all but two edges adjacent to a vertex are excluded
    #If so, these edges are included
    def addEdges(self, constraints):
        for i in range(self.size):
            t = 0
            for j in range(self.size):
                if constraints[i][j] == 0:
                    t += 1
            if t == self.size - 2:
                for j in range(self.size):
                    if constraints[i][j] == 2:
                        constraints[i][j] = 1
                        constraints[j][i] = 1
        return constraints
    
    #Determines whether there exists an included edge that starts in fr and does not end in prev
    #If so, it also returns the endpoint of this edge.
    def next_one (self, prev, fr, constraints):
        for j in range(self.size):
            if (constraints[fr][j] == 1) and (j != prev):
                return [True, j]
        return [False]
    
    #Determines if a node representd a full tour by checking whether from every vertex
    #there are exactly 2 included edges and all other edges are excluded
    def isTour(self):
        for i in range(self.size):
            num_zero = 0
            num_one = 0
            for j in range(self.size):
                if self.constraints[i][j] == 0:
                    num_zero += 1
                elif self.constraints[i][j] == 1:
                    num_one += 1
            if (num_zero != self.size - 2) or (num_one != 2):
                return False
        return True

    #Checks if a node contains a subtour
    def contains_subtour(self):
        for i in range(self.size):
            next = self.next_one(i, i, self.constraints)
            if next[0]:
                prev = i
                fr = next[1]
                t = 1
                next = self.next_one(prev, fr, self.constraints)
            while next[0]:
                t += 1
                prev = fr
                fr = next[1]
                if (fr == i) and (t < self.size):
                    return True
                next = self.next_one(prev, fr, self.constraints)
                if t == self.size:
                    return False
        return False
    
    #Assumes the node represents a tour and returns the length
    def tourLength(self):
        length = 0
        fr = 0
        print ("1: ", self.next_one(fr, fr, self.constraints))
        if self.next_one(fr, fr, self.constraints)[0]:
            print ("2: ", self.next_one(fr, fr, self.constraints))
            to = self.next_one(fr, fr, self.constraints)[1]
            for i in range(self.size):
                length += self.costs[fr][to]
                temp = fr
                fr = to
                if self.next_one(temp, to, self.constraints)[0]:
                    to = self.next_one(temp, to, self.constraints)[1]
        return length

    #Determines the next constraint according to the branching strategy
    #lexicographic order (LG)
    def next_constraint(self):
        for i in range(self.size):
            for j in range(self.size):
                if self.constraints[i][j] == 2:
                    return [i,j]
                
    #If a node represents a tour
    #returns a string with the order of the vertices in the tour
    def __str__(self):
        if self.isTour():
            result = "0"
            fr = 0
            to = None
            for j in range(self.size):
                if self.constraints[fr][j] == 1:
                    to = j
                    result += "−" + str(j)
                    break
            for i in range(self.size - 1):
                for j in range(self.size):
                    if (self.constraints[to][j] == 1) and (j != fr):
                        result += "−" + str(j)
                        fr = to
                        to = j
                        break
            return result
        else:
            print("This node is not a tour")

In [120]:
#from Node import Node
import math
import time
import copy

class TSP():
    
    def __init__(self, size, costs, bestTour=math.inf):
        self.size = size
        self.costs = costs
        self.bestTour = bestTour
        self.bestNode = None
        self.bestNodeTime = 0
        self.num_createdNodes = 0
        self.num_prunedNodes = 0
        self.sortedEdges = self.sort_edges()
        self.allSortedEdges = self.sort_allEdges()
        
    def findSolution(self):
        root = self.create_root()
        self.num_createdNodes += 1
        T1 = time.perf_counter()
        self.BranchAndBound(root)
        T2 = time.perf_counter()
        print("−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−−")
        print("The shortest tour is:", self.bestNode)
        print("It has a length of", self.bestTour, "km")
        print("Found in", T2-T1, "seconds")
        print("Best tour was found after:", self.bestNodeTime, "seconds")
        print("Number of nodes created: ", self.num_createdNodes)
        print("Number of nodes pruned: ", self. num_prunedNodes)
        
    #sorts the edges of the distance matrix per row returns matirix where each
    #row i contains the numbers 0 <= k <= (self.size-1) in increasing costs of the edges (i,k)
    def sort_edges(self):
        result = []
        for i in range(self.size):
            result.append([x for (y, x) in sorted(zip(self.costs[i], list(range(self.size))))])
        return result
    
    #sorts all edges of distance matrix
    #returns list of pairs [i,j] in order of increasing costs
    def sort_allEdges(self):
        edges = []
        lengths = []
        for i in range(self.size):
            for j in range(i+1, self.size):
                edges.append([i, j])
                lengths.append(self.costs[i][j])
        result = [z for (l, z) in sorted(zip(lengths, edges))]
        return result
    
    def create_root(self):
        no_constraints = []
        for i in range(self.size):
            row_i = []
            for j in range(self.size):
                if (i != j):
                    row_i.append(2)
                else:
                    row_i.append (0)
            no_constraints.append(row_i)
        root = Node(self.size, self.costs, self.sortedEdges, self.allSortedEdges, no_constraints)
        return root
    
    def BranchAndBound(self, node):
        if node.isTour():
            if node.tourLength() < self.bestTour:
                self.bestTour = node.tourLength()
                self.bestNode = node
                self.bestNodeTime = time.perf_counter()
                print("Found better tour: ", self.bestNode, "of length", self.bestTour, "km")
        else:
            new_constraint = copy.copy(node.next_constraint())
            new_constraint.append(1)
            leftChild = Node(self.size, self.costs, self.sortedEdges, self.allSortedEdges, node.constraints, new_constraint)
            new_constraint[2] = 0
            rightChild = Node(self.size, self.costs, self.sortedEdges, self.allSortedEdges, node.constraints, new_constraint)
            self.num_createdNodes += 2
            
            print(leftChild.lowerBound, leftChild.LP, "which one is bigger?")
            
            if self.num_createdNodes%401 == 0:
                print("Number of nodes created so far: ", self.num_createdNodes)
                print("Number of nodes pruned so far: ", self.num_prunedNodes)
            if self.num_createdNodes%51 == 0:
                print(".")
            if (leftChild.contains_subtour()) or (leftChild.lowerBound > 2 * self.bestTour):
                leftChild = None
                self.num_prunedNodes += 1
            if (rightChild.contains_subtour()) or (rightChild.lowerBound > 2 * self.bestTour):
                rightChild = None
                self.num_prunedNodes += 1
            if (leftChild != None) and (rightChild == None):
                self.BranchAndBound(leftChild)
            elif (leftChild == None) and (rightChild != None):
                self. BranchAndBound(rightChild)
            elif (leftChild != None) and (rightChild != None):
                if leftChild.lowerBound <= rightChild.lowerBound:
                    if leftChild.lowerBound < 2 * self.bestTour:
                        self.BranchAndBound(leftChild)
                    else:
                        leftChild = None
                        self.num_prunedNodes += 1
                    if rightChild.lowerBound < 2*self.bestTour:
                        self.BranchAndBound(rightChild)
                    else:
                        rightChild = None
                        self.num_prunedNodes += 1
                else:
                    if rightChild.lowerBound < 2*self.bestTour:
                        self.BranchAndBound(rightChild)
                    else:
                        rightChild = None
                        self.num_prunedNodes += 1
                    if leftChild.lowerBound < 2*self.bestTour:
                        self.BranchAndBound(leftChild)
                    else:
                        leftChild = None
                        self.num_prunedNodes += 1
       
    #Determines the next constraint using IL
    def next_constraint(self):
        for edge in self.allSortedEdges:
            i = edge[0]
            j = edge[1]
            if self.constraints[i][j] == 2:
                return edge

In [112]:
cost = [[0,141,118,171,126,69,158],
[141,0,226,34,212,208,82],
[118,226,0,232,56,107,194],
[171,34,232,0,200,233,63],
[126,212,56,200,0,105,145],
[69,208,107,233,105,0,212],
[158,82,194,63,145,212,0]]

In [116]:
T = TSP(7, cost)
T.findSolution()

Optimization terminated successfully.
         Current function value: 527.000000  
         Iterations: 20
[0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0.]
Optimization terminated successfully.
         Current function value: 527.000000  
         Iterations: 20
[0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0.]
Optimization terminated successfully.
         Current function value: 615.000000  
         Iterations: 18
[1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0.]
Optimization terminated successfully.
         Current function value: 615.000000  
         Iterations: 18
[1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0.]
Optimization terminated successfully.
         Current function value: 527.000000  
         Iterations: 17
[0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0.]
Optimization terminated successfully.
         Current function value: 527.000000  
         Iterations: 17
[0. 1. 0. 0. 1. 0. 0. 1. 0.

Optimization terminated successfully.
         Current function value: 664.000000  
         Iterations: 24
[0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0.]
Optimization terminated successfully.
         Current function value: 664.000000  
         Iterations: 24
[0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0.]
Optimization terminated successfully.
         Current function value: 537.000000  
         Iterations: 22
[0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0.]
Optimization terminated successfully.
         Current function value: 537.000000  
         Iterations: 22
[0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0.]
1328.0 1328.0 which one is bigger?
Optimization terminated successfully.
         Current function value: 537.000000  
         Iterations: 18
[0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0.]
Optimization terminated successfully.
         Current function value: 537.000000  
         Iterati

Optimization terminated successfully.
         Current function value: 725.000000  
         Iterations: 19
[1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0.]
Optimization terminated successfully.
         Current function value: 615.000000  
         Iterations: 23
[1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0.]
Optimization terminated successfully.
         Current function value: 615.000000  
         Iterations: 23
[1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0.]
1450.0 1450.0 which one is bigger?
Optimization terminated successfully.
         Current function value: 739.000000  
         Iterations: 24
[1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1.]
Optimization terminated successfully.
         Current function value: 739.000000  
         Iterations: 24
[1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1.]
Optimization terminated successfully.
         Current function value: 615.000000  
         Iterati

In [121]:
T = TSP(20, cost)
T.findSolution()

Optimization terminated successfully.
         Current function value: 362.000000  
         Iterations: 84
Optimization terminated successfully.
         Current function value: 362.000000  
         Iterations: 84
Optimization terminated successfully.
         Current function value: 418.000000  
         Iterations: 78
Optimization terminated successfully.
         Current function value: 418.000000  
         Iterations: 78
Optimization terminated successfully.
         Current function value: 362.000000  
         Iterations: 79
Optimization terminated successfully.
         Current function value: 362.000000  
         Iterations: 79
836.0 836.0 which one is bigger?
Optimization terminated successfully.
         Current function value: 373.000000  
         Iterations: 106
Optimization terminated successfully.
         Current function value: 373.000000  
         Iterations: 106
Optimization terminated successfully.
         Current function value: 362.000000  
         Iteratio

Optimization terminated successfully.
         Current function value: 418.000000  
         Iterations: 99
Optimization terminated successfully.
         Current function value: 364.500000  
         Iterations: 86
Optimization terminated successfully.
         Current function value: 364.500000  
         Iterations: 86
836.0 836.0 which one is bigger?
Optimization terminated successfully.
         Current function value: 405.000000  
         Iterations: 78
Optimization terminated successfully.
         Current function value: 405.000000  
         Iterations: 78
Optimization terminated successfully.
         Current function value: 364.500000  
         Iterations: 89
Optimization terminated successfully.
         Current function value: 364.500000  
         Iterations: 89
810.0 810.0 which one is bigger?
Optimization terminated successfully.
         Current function value: 383.000000  
         Iterations: 95
Optimization terminated successfully.
         Current function value:

Optimization terminated successfully.
         Current function value: 421.000000  
         Iterations: 90
Optimization terminated successfully.
         Current function value: 421.000000  
         Iterations: 90
Optimization terminated successfully.
         Current function value: 367.000000  
         Iterations: 94
Optimization terminated successfully.
         Current function value: 367.000000  
         Iterations: 94
841.9999999999999 841.9999999999999 which one is bigger?
Optimization terminated successfully.
         Current function value: 411.000000  
         Iterations: 84
Optimization terminated successfully.
         Current function value: 411.000000  
         Iterations: 84
Optimization terminated successfully.
         Current function value: 367.000000  
         Iterations: 83
Optimization terminated successfully.
         Current function value: 367.000000  
         Iterations: 83
822.0 822.0 which one is bigger?
Optimization terminated successfully.
        

Optimization terminated successfully.
         Current function value: 380.000000  
         Iterations: 82
816.0 816.0 which one is bigger?
Optimization terminated successfully.
         Current function value: 400.000000  
         Iterations: 108
Optimization terminated successfully.
         Current function value: 400.000000  
         Iterations: 108
Optimization terminated successfully.
         Current function value: 380.000000  
         Iterations: 83
Optimization terminated successfully.
         Current function value: 380.000000  
         Iterations: 83
800.0 800.0 which one is bigger?
Optimization terminated successfully.
         Current function value: 414.000000  
         Iterations: 88
Optimization terminated successfully.
         Current function value: 414.000000  
         Iterations: 88
Optimization terminated successfully.
         Current function value: 380.000000  
         Iterations: 86
Optimization terminated successfully.
         Current function valu

Optimization terminated successfully.
         Current function value: 416.500000  
         Iterations: 64
Optimization terminated successfully.
         Current function value: 416.500000  
         Iterations: 64
906.0 906.0 which one is bigger?
Optimization terminated successfully.
         Current function value: 458.300000  
         Iterations: 90
Optimization terminated successfully.
         Current function value: 458.300000  
         Iterations: 90
Optimization terminated successfully.
         Current function value: 416.500000  
         Iterations: 61
Optimization terminated successfully.
         Current function value: 416.500000  
         Iterations: 61
916.5999999999999 916.5999999999999 which one is bigger?
Optimization terminated successfully.
         Current function value: 463.000000  
         Iterations: 60
Optimization terminated successfully.
         Current function value: 463.000000  
         Iterations: 60
Optimization terminated successfully.
        

...

Optimization terminated successfully.
         Current function value: 380.000000  
         Iterations: 87
816.0 816.0 which one is bigger?
Optimization terminated successfully.
         Current function value: 413.000000  
         Iterations: 72
Optimization terminated successfully.
         Current function value: 413.000000  
         Iterations: 72
Optimization terminated successfully.
         Current function value: 380.000000  
         Iterations: 95
Optimization terminated successfully.
         Current function value: 380.000000  
         Iterations: 95
826.0 826.0 which one is bigger?
Optimization terminated successfully.
         Current function value: 428.000000  
         Iterations: 89
Optimization terminated successfully.
         Current function value: 428.000000  
         Iterations: 89
Optimization terminated successfully.
         Current function value: 380.000000  
         Iterations: 88
Optimization terminated successfully.
         Current function value:

Optimization terminated successfully.
         Current function value: 382.500000  
         Iterations: 107
Optimization terminated successfully.
         Current function value: 382.500000  
         Iterations: 107
784.0 784.0 which one is bigger?
Optimization terminated successfully.
         Current function value: 401.000000  
         Iterations: 129
Optimization terminated successfully.
         Current function value: 401.000000  
         Iterations: 129
Optimization terminated successfully.
         Current function value: 382.500000  
         Iterations: 110
Optimization terminated successfully.
         Current function value: 382.500000  
         Iterations: 110
802.0 802.0 which one is bigger?
Optimization terminated successfully.
         Current function value: 388.000000  
         Iterations: 117
Optimization terminated successfully.
         Current function value: 388.000000  
         Iterations: 117
Optimization terminated successfully.
         Current functio

Optimization terminated successfully.
         Current function value: 383.000000  
         Iterations: 86
Optimization terminated successfully.
         Current function value: 383.000000  
         Iterations: 86
826.0 826.0 which one is bigger?
Optimization terminated successfully.
         Current function value: 449.000000  
         Iterations: 94
Optimization terminated successfully.
         Current function value: 449.000000  
         Iterations: 94
Optimization terminated successfully.
         Current function value: 383.000000  
         Iterations: 92
Optimization terminated successfully.
         Current function value: 383.000000  
         Iterations: 92
898.0 898.0 which one is bigger?
Optimization terminated successfully.
         Current function value: 383.000000  
         Iterations: 89
Optimization terminated successfully.
         Current function value: 383.000000  
         Iterations: 89
Optimization terminated successfully.
         Current function value:

Optimization terminated successfully.
         Current function value: 384.000000  
         Iterations: 97
Optimization terminated successfully.
         Current function value: 384.000000  
         Iterations: 97
940.0 940.0 which one is bigger?
Optimization terminated successfully.
         Current function value: 438.000000  
         Iterations: 102
Optimization terminated successfully.
         Current function value: 438.000000  
         Iterations: 102
Optimization terminated successfully.
         Current function value: 384.000000  
         Iterations: 84
Optimization terminated successfully.
         Current function value: 384.000000  
         Iterations: 84
876.0 876.0 which one is bigger?
Optimization terminated successfully.
         Current function value: 414.000000  
         Iterations: 108
Optimization terminated successfully.
         Current function value: 414.000000  
         Iterations: 108
Optimization terminated successfully.
         Current function va

Number of nodes pruned:  3981


In [17]:
TSP.sort_edges

<function __main__.TSP.sort_edges(self)>

In [58]:
cost = [[0, 10, 15, 20],
       [10, 0, 35, 25],
       [15, 35, 0, 30],
       [20, 25, 30, 0]]

In [117]:
file = open("twenty.tsp", "r")
coordinates = np.genfromtxt(file, dtype = float, skip_header = 7, usecols = (1,2))

cost = np.array([[0]*len(coordinates)]*len(coordinates))
sortededges = np.array([[0]*len(coordinates)]*len(coordinates))
allsortededges = []
parentconstraint = np.array([[2]*len(coordinates)]*len(coordinates))

for i in range(len(coordinates)):
    for j in range(len(coordinates)):
        if i != j:
            cost[i][j] = np.sqrt(((coordinates[i][0]-coordinates[j][0])**2 + (coordinates[i][1]-coordinates[j][1])**2))
            
        if i > j:
            allsortededges += [cost[i][j]]
            
for k in range(len(coordinates)):
    sortededges[k] = sorted(cost[k])
        
allsortededges = sorted(allsortededges)
print (cost)

[[  0  84  51  81 111  67  31  55 117  47  62  69  79  90  26  54  44  71
   69  60]
 [ 84   0  59  23  28  65  55  34  33  37  22  16  51   6  66  29  44  17
   51  65]
 [ 51  59   0  71  87  88  26  54  84  44  47  54  88  65  56  42  22  57
   19  82]
 [ 81  23  71   0  35  44  58  25  49  34  24  17  29  24  58  30  50  13
   69  46]
 [111  28  87  35   0  79  84  58  18  63  48  42  60  21  91  56  72  40
   76  81]
 [ 67  65  88  44  79   0  64  36  94  44  51  51  20  68  42  50  65  48
   94   7]
 [ 31  55  26  58  84  64   0  36  87  25  36  43  68  62  29  28  13  46
   40  58]
 [ 55  34  54  25  58  36  36   0  68  10  15  18  33  39  33  13  32  17
   59  34]
 [117  33  84  49  18  94  87  68   0  71  55  50  77  28 100  62  74  50
   71  96]
 [ 47  37  44  34  63  44  25  10  71   0  15  21  43  43  29   9  21  23
   50  40]
 [ 62  22  47  24  48  51  36  15  55  15   0   7  44  28  44   7  27  10
   47  49]
 [ 69  16  54  17  42  51  43  18  50  21   7   0  41  22  50  14

In [180]:
#Brute force algorithm
import itertools
import math

n = 20
distances = cost
minLength = math.inf
minTour = []

for tour in itertools.permutations(list(range(1, n))):
    fr = 0
    length = 0
    count = 0
    while count < n - 1:
        to = tour[count]
        length += distances[fr][to]
        fr = to
        count += 1
    length += distances[fr][0]
    if length < minLength:
        minLength = length
        minTour = tour
minTour = (0,) + minTour + (0,)
print("Shortest tour is: ", minTour)
print("It has a length of: ", minLength , "km")

KeyboardInterrupt: 